# Pabbi Land Cover Classification Project

In [1]:
import ee
import geemap

In [2]:
geemap.ee.Initialize()


In [5]:
url = 'https://github.com/code4geoai/gee/releases/download/0.1/pabbigeojson.geojson'
#url = "https://github.com/code4geoai/gee/releases/download/0.1/Pabbi_Mouzas.zip"
m = geemap.Map()
fc = geemap.geojson_to_ee(url)
#style = {'color': 'red', 'width': 2}
#fc = geemap.shp_to_ee (url)
#m.add_layer(fc, {}, 'Pabbi Mouzas')
#m.center_object(fc, 10)
#m

Downloading...
From: https://github.com/code4geoai/gee/releases/download/0.1/pabbigeojson.geojson
To: C:\Users\USER\AppData\Local\Temp\24ba2a8a-73f5-4465-af50-9346dcc07fcb.geojson
100%|██████████| 4.57M/4.57M [00:04<00:00, 969kB/s] 


In [6]:
geemap.geojson_to_ee(url).first().getInfo()

Downloading...
From: https://github.com/code4geoai/gee/releases/download/0.1/pabbigeojson.geojson
To: C:\Users\USER\AppData\Local\Temp\9eba57ab-7c7a-465e-aa09-4beb18e6f478.geojson
100%|██████████| 4.57M/4.57M [00:11<00:00, 409kB/s]


{'type': 'Feature',
 'geometry': {'geodesic': False,
  'type': 'MultiPolygon',
  'coordinates': [[[[753923.0771000003, 3769111.1411000006],
     [753894.5981000001, 3769108.1548999995],
     [753894.5959999999, 3769123.1240999997],
     [753917.5783000002, 3769127.4113],
     [753923.0771000003, 3769111.1411000006]]]]},
 'id': '0',
 'properties': {'Area_Acre': 0.101785,
  'FFID': 1,
  'Landuse_Ma': 'Agriculture',
  'Mouza_Name': 'Khushmaqam',
  'Parcel_ID': 668}}

In [50]:
import geopandas as gpd

filepath = "D:\Trainings\Pabbi_Mouzas\pabbigeojson.geojson"
gdf = gpd.read_file(filepath)
pabbi= geemap.gdf_to_ee(gdf)


In [51]:
# Applying filters on the features
gdf.head()
# Counting Landuse_Ma field having value Agriculture
gdf['Landuse_Ma'].value_counts()

Landuse_Ma
Agriculture     5759
Stream           302
Other            229
Road/Streets     172
Graveyard         29
Built up          24
Name: count, dtype: int64

In [52]:

#Filtering the features having Landuse_Ma as Builtup
builtup=gdf[gdf['Landuse_Ma'] == 'Built up']

builtstyle = {'color': 'red', 
         'width': 1,
         'fillColor': '00000000',
            
         }

In [53]:
#Filtering the features having Landuse_Ma as Agriculture
agri=gdf[gdf['Landuse_Ma'] == 'Agriculture']

agristyle = {'color': 'pink', 
         'width': 1,
         'fillColor': '00000000',
            
         }

In [54]:
#Filtering the features having Landuse_Ma as Graveyard
graveyard=gdf[gdf['Landuse_Ma'] == 'Graveyard']

gravestyle = {'color': 'black', 
         'width': 1,
         'fillColor': '00000000',
            
         }

In [55]:
m1 = geemap.Map()
style = {'color': 'blue', 
         'width': 1,
         'fillColor': '00000000',
             
         }
m1.add_basemap('HYBRID')
m1.addLayer(pabbi.style(**style), {}, 'Pabbi Mouzas')
m1.add_gdf(builtup, layer_name='Built up', style=builtstyle)
m1.add_gdf(agri, layer_name='Agriculture', style=agristyle)
m1.add_gdf(graveyard, layer_name='Graveyard', style=gravestyle)
m1.center_object(pabbi, 10)
m1

Map(center=[34.02510458074524, 71.77439125564787], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:

style = {'color': 'blue', 
         'width': 1,
         'fillColor': '00000000',
             
         }

In [63]:
p = geemap.Map()
p.add_basemap('HYBRID')

collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
# Filter the collection for a specific region and time period
start_date = '2024-01-01'
end_date = '2024-12-31'
# Filter the collection by date and region
filtered_collection = collection.filterDate(start_date, end_date).filterBounds(pabbi.geometry()).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# Select the first image from the filtered collection
median_image = filtered_collection.median().clip(pabbi.geometry())

# Calculate NDVI
ndvi = median_image.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Select the bands of interest (e.g.,NVDI)
ndvi_vis = {
    'min': 0.25,
    'max': 0.8,
    'palette': ['white', 'yellow', 'green', 'red']  # Red for highest vegetation
}

# Add the image to the map
p.addLayer(ndvi, ndvi_vis,  name='Median NDVI')

p.add_colorbar(
    vis_params=ndvi_vis,
    label='NDVI',
    orientation='horizontal',
    position='bottomright',
    layer_name = 'Median NDVI',
    
)

p.addLayer(pabbi.style(**style), {}, 'Pabbi Mouzas')
p.add_gdf(builtup, layer_name='Built up', style=builtstyle)
p.add_gdf(agri, layer_name='Agriculture', style=agristyle)
p.add_gdf(graveyard, layer_name='Graveyard', style=gravestyle)



p.center_object(pabbi, 10)

p



Map(center=[34.02510458074524, 71.77439125564787], controls=(WidgetControl(options=['position', 'transparent_b…

# Selecting only those parcels which have ndvi value between .3 and 0.7 

In [64]:
# Step 1: Threshold NDVI values
ndvi_mask = ndvi.gt(0.3).And(ndvi.lt(0.7))

# Step 2: Convert boolean mask to integer (1s and 0s)
ndvi_mask_int = ndvi_mask.updateMask(ndvi_mask).multiply(1).toInt()

# Step 3: Convert NDVI mask to vector polygons
ndvi_vector = ndvi_mask_int.reduceToVectors(
    geometry=pabbi.geometry(),
    scale=10,
    geometryType='polygon',
    labelProperty='ndvi_class',
    maxPixels=1e13
)

# Step 4: Spatial join - filter pabbi polygons that intersect with NDVI areas
pabbi_ndvi_filtered = pabbi.filterBounds(ndvi_vector.geometry())

# Style the filtered pabbi
highlight_style = {'color': 'blue', 'fillColor': '00000000'}

# Add to map
p.addLayer(ndvi_mask_int, {'min': 0, 'max': 1, 'palette': ['white', 'green'],'opacity':0.4}, 'NDVI Mask 0.3–0.7')
#p.addLayer(pabbi_ndvi_filtered.style(**highlight_style), {}, 'Pabbi in NDVI Range')


In [65]:
p

Map(bottom=1675472.0, center=[34.022786817002, 71.78328837919378], controls=(WidgetControl(options=['position'…